In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os


repo_path = '.'

if repo_path not in sys.path:
    sys.path.append(repo_path)

# Set up the cache for this notebook
os.environ["DSP_NOTEBOOK_CACHEDIR"] = os.path.join(repo_path, 'cache')


In [ ]:
!jupyter nbextension enable --py widgetsnbextension

In [2]:
import pkg_resources # Install the package if it's not installed
if not "dspy-ai" in {pkg.key for pkg in pkg_resources.working_set}:
    !pip install -U pip
    !pip install dspy-ai
    !pip install openai~=0.28.1
    # !pip install -e $repo_path

import dspy

In [3]:
import configparser
config = configparser.ConfigParser()

# Read the configuration file
config.read('config.ini')

# Access the API key
api_key = config['openai']['api_key']


There's a hosted server with COLBERT that includes wikipedia extracts. We'll use it for the "retriever model"

In [4]:
turbo = dspy.OpenAI(model='gpt-3.5-turbo', api_key=api_key)
colbertv2_wiki17_abstracts = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')
dspy.settings.configure(lm=turbo, rm=colbertv2_wiki17_abstracts)


In [20]:
# prove that openai is working

from openai import OpenAI

client = OpenAI(api_key=api_key)

# Make a call to GPT-3.5 Turbo to test the API key
try:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "What is the capital of California?"
            }
        ]
    )
    print("Success! Here's the response from GPT-3.5 Turbo:")
    print(response.choices[0].message.content)
except Exception as e:
    print("An error occurred:")
    print(e)

Success! Here's the response from GPT-3.5 Turbo:
The capital of California is Sacramento.


### How DSPy programming works

#### A word on the workflow

You can build your own DSPy programs for various tasks, e.g., question answering, information extraction, or text-to-SQL.

Whatever the task, the general workflow is:

1. **Collect a little bit of data.** Define examples of the inputs and outputs of your program (e.g., questions and their answers). This could just be a handful of quick examples you wrote down. If large datasets exist, the more the merrier!
1. **Write your program.** Define the modules (i.e., sub-tasks) of your program and the way they should interact together to solve your task.
1. **Define some validation logic.** What makes for a good run of your program? Maybe the answers need to have a certain length or stick to a particular format? Specify the logic that checks that.
1. **Compile!** Ask DSPy to compile your program using your data. The compiler will use your data and validation logic to optimize your program (e.g., prompts and modules) so it's efficient and effective!
1. **Iterate.** Repeat the process by improving your data, program, validation, or by using more advanced features of the DSPy compiler.

In [5]:
from dspy.datasets import HotPotQA

# Load the dataset.
dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=0)

# Tell DSPy that the 'question' field is the input. Any other fields are labels and/or metadata.
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

len(trainset), len(devset)

/Users/taitlarson/.pyenv/versions/3.11.8/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


(20, 50)

DSPy typically requires very minimal labeling. Whereas your pipeline may involve six or seven complex steps, **you only need labels for the initial question and the final answer**. DSPy will bootstrap any intermediate labels needed to support your pipeline. If you change your pipeline in any way, the data bootstrapped will change accordingly!

In [9]:
train_example = trainset[0]
print(f"Question: {train_example.question}")
print(f"Answer: {train_example.answer}")

Question: At My Window was released by which American singer-songwriter?
Answer: John Townes Van Zandt


Examples in the dev set contain a third field, namely, `titles` of relevant Wikipedia articles. This is not essential but, for the sake of this intro, it'll help us get a sense of how well our programs are doing.

In [10]:
dev_example = devset[18]
print(f"Question: {dev_example.question}")
print(f"Answer: {dev_example.answer}")
print(f"Relevant Wikipedia Titles: {dev_example.gold_titles}")

Question: What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?
Answer: English
Relevant Wikipedia Titles: {'Restaurant: Impossible', 'Robert Irvine'}


### Exploring labels and valid outputs

After loading the raw data, we'd applied x.with_inputs('question') to each example to tell DSPy that our input field in each example will be just `question`. Any other fields are labels or metadata that are not given to the system.

In [11]:
print(f"For this dataset, training examples have input keys {train_example.inputs().keys()} and label keys {train_example.labels().keys()}")
print(f"For this dataset, dev examples have input keys {dev_example.inputs().keys()} and label keys {dev_example.labels().keys()}")

For this dataset, training examples have input keys ['question'] and label keys ['answer']
For this dataset, dev examples have input keys ['question'] and label keys ['answer', 'gold_titles']


In DSPy, we will maintain a clean separation between defining your modules in a **declarative way** and calling them in a **pipeline** to solve the task.

## Signatures

Every call to the LM in a DSPy program needs to have a Signature.

A signature consists of three simple elements:

* A minimal description of the **sub-task** the LM is supposed to solve.
* A description of one or more **input fields** (e.g., input question) that will we will give to the LM.
* A description of one or more **output fields** (e.g., the question's answer) that we will expect from the LM.

In [12]:
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers"""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")


In `BasicQA`, the docstring describes the sub-task here (i.e., answering questions). Each `InputField` or `OutputField` can optionally contain a description `desc` too. When it's not given, it's inferred from the field's name (e.g., `question`)

## Predictors

A predictor is a module that knows how to use the LM to implement a signature. Importantly, predictors can **learn** to fit their behavior to the task!

In [21]:
# Define the predictor
generate_answer = dspy.Predict(BasicQA)

# call the predictor on some input
pred = generate_answer(question=dev_example.question)

# Print the input and the prediction.
print(f"Question: {dev_example.question}")
print(f"Predicted Answer: {pred.answer}")

Question: What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?
Predicted Answer: American
